In [ ]:
from typing import *
import torch
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
from torch import nn

In [ ]:
from python.configuration_llama import CustomLlamaConfig
from python.modeling_llama import CustomLlamaForCausalLM

In [ ]:
original_config = AutoConfig.from_pretrained('../../base_model/Meta-Llama-3-8B-hf')
original_model = AutoModelForCausalLM.from_pretrained('../../base_model/Meta-Llama-3-8B-hf', torch_dtype=original_config.torch_dtype)

In [ ]:
config = CustomLlamaConfig().from_pretrained('.')

In [ ]:
config.activation_sparsity_type = None
config.use_spvmm = False
config.use_vmmsp = False
config.use_spvmm_cpu = False
config.use_vmmsp_cpu = False

In [ ]:
config

In [ ]:
model = CustomLlamaForCausalLM(config)

In [ ]:
model.to(config.torch_dtype)

In [ ]:
model.load_state_dict(original_model.state_dict(), strict=False)

In [ ]:
for original_layer, layer in zip(original_model.model.layers, model.model.layers):
    layer.self_attn.q_proj_weight_t = nn.Parameter(original_layer.self_attn.q_proj.weight.transpose(0,1).contiguous())
    layer.self_attn.o_proj_weight_t = nn.Parameter(original_layer.self_attn.o_proj.weight.transpose(0, 1).contiguous())
    layer.mlp.down_proj_weight_t = nn.Parameter(original_layer.mlp.down_proj.weight.transpose(0, 1).contiguous())

In [ ]:
thresholds = torch.load('../thresholds_0_5.pt')

In [ ]:
thresholds.keys()

In [ ]:
for layer_idx, layer in enumerate(model.model.layers):
    layer.mlp.gate_proj_states_thresholds.data = thresholds['gate_proj_states_thresholds'][layer_idx].to(config.torch_dtype)
    layer.mlp.gate_proj_states_thresholds_2.data = thresholds['gate_proj_states_thresholds_2'][layer_idx].to(config.torch_dtype)
    layer.self_attn.attention_inputs_thresholds.data = thresholds['attention_inputs_thresholds'][layer_idx].to(config.torch_dtype)
    layer.self_attn.attention_outputs_thresholds.data = thresholds['attention_outputs_thresholds'][layer_idx].to(config.torch_dtype)

In [ ]:
model.state_dict()

In [ ]:
config.register_for_auto_class()
model.register_for_auto_class(AutoModelForCausalLM)

In [ ]:
config.save_pretrained('./model')
model.save_pretrained('./model')

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('../../base_model/Meta-Llama-3-8B-hf')
tokenizer.save_pretrained('./model')